# Assignment 2 - Applied Genomics 
## Lauren Shelby 

## Part 0

In [2]:
pwd

/scratch/las10014/ApGenomics2024/Homework2


In [2]:
# Making new directory for this assignment and changing directories into that folder 

mkdir Homework2
cd Homework2

In [7]:
#Copying files needed into this folder

cp /scratch/work/courses/AppliedGenomicsSec3/students/Homework2/ce10.fa ./
cp /scratch/work/courses/AppliedGenomicsSec3/students/Homework2/ce10.refGene.gtf ./
cp /scratch/work/courses/AppliedGenomicsSec3/students/Homework2/HWVLJBGXT_n01_HW16.fastq ./

## Part 1

### BWA-mem Alignment 

In [22]:
# Checking the correct synthax of BWA-mem module 

module avail bwa


--------------------------- /share/apps/modulefiles ----------------------------
   bwa-mem2/2.1    bwa/intel/0.7.17


In [25]:
cat BWA-mem_alignment.txt

#!/bin/bash
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --time=1:00:00
#SBATCH --mem=32GB
#SBATCH --job-name=ce10_BWA-mem__ce10_alignment

# Purging and loading the BWA-mem module 
module purge 
module load bwa/intel/0.7.17

# Using bwa index to create the index for alignment 
bwa index ce10.fa 

# Using bwa mem to run the alignment 
bwa mem ce10.fa HWVLJBGXT_n01_HW16.fastq > ce10_alignment_bwa.sam

In [26]:
sbatch BWA-mem_alignment.txt

Submitted batch job 42924251


### HISAT2 Alignment

In [27]:
# Checking for HISAT2 module 

module avail hisat


--------------------------- /share/apps/modulefiles ----------------------------
   hisat-3n/20231120    hisat2/2.2.1


In [28]:
cat HISAT2_alignment.txt

#!/bin/bash
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --time=1:00:00
#SBATCH --mem=32GB
#SBATCH --job-name=HISAT2_ce10_alignment

# Purging and loading the HISAT2 module 
module purge 
module load hisat2/2.2.1

# Running hisat2_extract_exons.py to extract information about exons 
hisat2_extract_exons.py ce10.refGene.gtf > ce10_exons.txt

# Running hisat2_extract_splice_sites.py to extract information about splice sites 
hisat2_extract_splice_sites.py ce10.refGene.gtf > ce10_splicesites.txt 

# Using hisat2-build in combination with ce10_exons.txt and ce10_splicesites.txt to build the index 
hisat2-build ce10.fa \
--ss ce10_splicesites.txt \
--exon ce10_exons.txt \
index_hisat2

# Running the alignment with hisat2 
hisat2 -x index_hisat2 -U HWVLJBGXT_n01_HW16.fastq -S ce10_alignment_hisat2.sam --rna-strandness R


In [29]:
sbatch HISAT2_alignment.txt

Submitted batch job 42924426


### STAR Alignment 

In [31]:
# Checking for STAR module 

module avail star


--------------------------- /share/apps/modulefiles ----------------------------
   star/intel/2.7.6a    star/intel/2.7.11a    starpu/openmpi/intel/20230822


In [1]:
cat STAR_alignment.txt

#!/bin/bash
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --time=1:00:00
#SBATCH --mem=32GB
#SBATCH --job-name=STAR_ce10_alignment

# Purging and loading the STAR module 
module purge 
module load star/intel/2.7.11a

# Building the STAR index 
STAR --runThreadN 2 \
--genomeSAindexNbases 12 \
--runMode genomeGenerate \
--genomeDir star_index \
--genomeFastaFiles ce10.fa \
--sjdbGTFfile ce10.refGene.gtf \
--sjdbOverhang 74

# Running the STAR alignment 
STAR --runThreadN 2 \
--readFilesIn HWVLJBGXT_n01_HW16.fastq \
--genomeDir star_index \
--outFileNamePrefix ce10_alignment_star

In [2]:
sbatch STAR_alignment.txt

Submitted batch job 42925714


## Part 2

### Question 1

In [5]:
# Checking the availability of the samtools module 
module avail sam


--------------------------- /share/apps/modulefiles ----------------------------
   libsamplerate/intel/0.2.1    samclip/0.4.0          samtools/intel/1.14
   pysam/0.16.0.1               samtools/intel/1.11
   samblaster/0.1.26            samtools/intel/1.12


In [6]:
# Purging and loading the samtools module 
module purge
module load samtools/intel/1.14

In [9]:
# Loading all files with 'sam' in this directory and displaying the 
    # execution permissions, file size, and displaying by time it was created
# The sam files are 5.1GB, 5.4GB, and 6.3GB respectively
ls -lht | grep sam

-rw-rw-r--. 1 las10014 las10014  5.4G Feb 13 17:06 ce10_alignment_starAligned.out.sam
-rw-rw-r--. 1 las10014 las10014  5.1G Feb 13 16:40 ce10_alignment_bwa.sam
-rw-rw-r--. 1 las10014 las10014  6.3G Feb 13 16:40 ce10_alignment_hisat2.sam


In [8]:
# Creating a loop for every file ending in 'sam' in this directory, to run 
    # the samtools module to convert it to a bam file 

for file in ./*sam
do
samtools view -bS $file > ${file::-3}bam
done

In [10]:
# Loading all 'bam' files in the directory created by the loop above 
    # Displaying the execution permissions, the size of the file, and sorted by time
    # The sam fils are 1.3GB, 1.4GB, and 1.4GB respectively, much smaller than the SAM files

ls -lht|grep bam

-rw-rw-r--. 1 las10014 las10014  1.3G Feb 14 09:26 ce10_alignment_starAligned.out.bam
-rw-rw-r--. 1 las10014 las10014  1.4G Feb 14 09:22 ce10_alignment_hisat2.bam
-rw-rw-r--. 1 las10014 las10014  1.4G Feb 14 09:18 ce10_alignment_bwa.bam


### Question 2

In [23]:
# Using samtools flagstat to get mapping statistics of the bam files 
    # generated by the 3 aligners 

samtools flagstat ce10_alignment_bwa.bam
samtools flagstat ce10_alignment_hisat2.bam
samtools flagstat ce10_alignment_starAligned.out.bam

20781509 + 0 in total (QC-passed reads + QC-failed reads)
19912446 + 0 primary
0 + 0 secondary
869063 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
20411339 + 0 mapped (98.22% : N/A)
19542276 + 0 primary mapped (98.14% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
22325177 + 0 in total (QC-passed reads + QC-failed reads)
19912446 + 0 primary
2412731 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
21407293 + 0 mapped (95.89% : N/A)
18994562 + 0 primary mapped (95.39% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
22172399 + 0 in total (QC-passed re

It looks like BWA aligned about 98% of the C. elegans genome to the reference, HISAT2 mapped about 95% of the C. elegans genome to the reference, and STAR mapped 100% of the C. elegans genome to the reference. 

BWA is good for long-read sequences and allows for more SNPs while aligning, but it doesn't take into account any sort of gene annnotation for aligning which makes me question the accuracy. 

HISAT2 takes into account splice site, adaptor sequences, and exon information which makes me believe it is more accurate than BWA. It can process both DNA and RNA sequences. 

STAR is designed specifically for RNA-seq data, which means it has awareness of splice sites, exons, and genes. In this context, I'd likely select STAR because we are using C. elegans RNA-seq data, and the samtools flagstat data show that 100% of the RNA-seq data was aligned to the reference. 